## Agent-Lab: Azure Entra ID Voice Memos Agent

### Setup

1. Login using az-cli:

```bash
az login --use-device-code
```

2. Create [App Registration](https://portal.azure.com/#view/Microsoft_AAD_RegisteredApps/ApplicationsListBlade) with the name `agent-lab`

3. Adjust [App Registration API Permissions](https://github.com/bsantanna/agent-lab/blob/main/doc/azure_entra_id_voice_memos_agent/app_registration_permissions.png)

4. Obtain AZURE_CLIENT_ID:

```bash
export AZURE_CLIENT_ID=$(az ad app list --display-name agent-lab --query "[0].appId" -o tsv)
```

5. Obtain AZURE_CLIENT_SECRET:

```bash
export AZURE_CLIENT_SECRET=$(az ad app credential reset --id ${AZURE_CLIENT_ID} --query "password" -o tsv)
```

6. Obtain AZURE_TENANT_ID:

```bash
export AZURE_TENANT_ID=$(az account show --query "tenantId" -o tsv)
```

7. Create a User

```bash
az ad user create \
   --display-name "Aline Real Madre" \
   --password "???" \
   --user-principal-name "a.real.madre@mailbsantanna.onmicrosoft.com" \
   --mail-nickname "a.real.madre"
```

8. Verify user:  [entra.microsoft.com](https://entra.microsoft.com/#view/Microsoft_AAD_UsersAndTenants/UserManagementMenuBlade/~/AllUsers/menuId/)





In [ ]:
%%capture

import json
import os

os.chdir("..")

from IPython.display import Markdown, display, Audio

from dotenv import load_dotenv

load_dotenv()

from notebooks import experiment_utils
from app.core.container import Container
from app.interface.api.messages.schema import MessageRequest

import nest_asyncio

nest_asyncio.apply()

# start dependency injection container
container = Container()
container.init_resources()
container.wire(modules=[__name__])

# get checkpointer instance
graph_persistence_factory = container.graph_persistence_factory()
checkpointer = graph_persistence_factory.build_checkpoint_saver()

---
### Upload Audio Attachment

In [ ]:
# create attachment
file_path = "tests/integration/voice_memos_01_pt_BR.mp3"
attachment_id = experiment_utils.create_attachment(
    file_path=file_path, content_type="audio/mp3"
)
display(Audio(file_path))

---
### OpenAI Voice Assistant

In [ ]:
# Create Workflow
openai_agent = experiment_utils.create_openai_agent(
    agent_type="azure_entra_id_voice_memos", llm_tag="gpt-4o-mini", api_key=os.getenv("OPENAI_API_KEY")
)
openai_voice_assistant = container.voice_memos_agent()
openai_workflow_builder = openai_voice_assistant.get_workflow_builder(openai_agent["id"])
openai_workflow = openai_workflow_builder.compile(checkpointer=checkpointer)
experiment_utils.print_graph(openai_workflow)

In [ ]:
%%capture

message = MessageRequest(
    message_role="human",
    message_content=(
        "Analyse the given audio."
    ),
    agent_id=openai_agent["id"],
    attachment_id=attachment_id,
)

inputs = openai_voice_assistant.get_input_params(message, schema="public")
config = openai_voice_assistant.get_config(openai_agent["id"])
result = openai_workflow.invoke(inputs, config)
ai_message_content, workflow_state = openai_voice_assistant.format_response(result)

In [ ]:
display(
    Markdown(
        f"**Execution Plan:**\n```json\n{json.dumps(workflow_state.get("execution_plan"), indent=2)}\n```"
    )
)

In [ ]:
display(Markdown(f"**AI Message Content:**\n\n{ai_message_content}"))